In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [15]:
from efficientnet.tfkeras import EfficientNetB4
from efficientnet.tfkeras import center_crop_and_resize, preprocess_input
from tensorflow.keras.losses import binary_crossentropy
import tensorflow as tf
import numpy as np
import utils_tf1
from glob import glob
import cv2

In [3]:
from tensorflow.keras.layers import concatenate

def convolution_block(x, filters, size, strides=(1,1), padding='same', activation=True):
    x = tf.keras.layers.Conv2D(filters, size, strides=strides, padding=padding)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    if activation == True:
        x = tf.keras.layers.LeakyReLU(alpha=0.1)(x)
    return x

def residual_block(blockInput, num_filters=16):
    x = tf.keras.layers.LeakyReLU(alpha=0.1)(blockInput)
    x = tf.keras.layers.BatchNormalization()(x)
    blockInput = tf.keras.layers.BatchNormalization()(blockInput)
    x = convolution_block(x, num_filters, (3,3) )
    x = convolution_block(x, num_filters, (3,3), activation=False)
    x = tf.math.add(x, blockInput)
    return x

def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = tf.keras.layers.Flatten()(y_true)
    y_pred_f = tf.keras.layers.Flatten()(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * tf.reduce_sum(intersection) + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)
    return 1. - score

def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

class Model:
    def __init__(self, img_size = 256, dropout_rate=0.1):
        self.X = tf.placeholder(tf.float32, (None, img_size, img_size, 3))
        self.Y = tf.placeholder(tf.float32, (None, img_size, img_size, 1))
        backbone = EfficientNetB4(weights='imagenet',
                            include_top=False,
                          input_tensor=self.X)
        self.efficientnet = backbone
        start_neurons = 16
        
        # [B, 16, 16, 160]
        conv4 = backbone.layers[314].output
        conv4 = tf.keras.layers.LeakyReLU(alpha=0.1)(conv4)
        pool4 = tf.keras.layers.MaxPooling2D((2, 2))(conv4)
        pool4 = tf.keras.layers.Dropout(dropout_rate)(pool4)
        # [B, 8, 8, 160]
        
        # [B, 8, 8, 512]
        convm = tf.keras.layers.Conv2D(start_neurons * 32, (3, 3), activation=None, padding="same")(pool4)
        convm = residual_block(convm,start_neurons * 32)
        convm = residual_block(convm,start_neurons * 32)
        # [B, 8, 8, 512]
        convm = tf.keras.layers.LeakyReLU(alpha=0.1)(convm)
        
        # [B, 16, 16, 256]
        deconv4 = tf.keras.layers.Conv2DTranspose(start_neurons * 16, (3, 3), strides=(2, 2), padding="same")(convm)
        uconv4 = concatenate([deconv4, conv4])
        # [B, 16, 16, 416]
        uconv4 = tf.keras.layers.Dropout(dropout_rate)(uconv4)
        
        # [B, 16, 16, 256]
        uconv4 = tf.keras.layers.Conv2D(start_neurons * 16, (3, 3), activation=None, padding="same")(uconv4)
        uconv4 = residual_block(uconv4,start_neurons * 16)
        uconv4 = residual_block(uconv4,start_neurons * 16)
        # [B, 16, 16, 256]
        uconv4 = tf.keras.layers.LeakyReLU(alpha=0.1)(uconv4)
        
        # [B, 32, 32, 128]
        deconv3 = tf.keras.layers.Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(uconv4)
        # [B, 32, 32, 56]
        conv3 = backbone.layers[138].output
        # [B, 32, 32, 184]
        uconv3 = concatenate([deconv3, conv3])    
        uconv3 = tf.keras.layers.Dropout(dropout_rate)(uconv3)
        
        # [B, 32, 32, 128]
        uconv3 = tf.keras.layers.Conv2D(start_neurons * 8, (3, 3), activation=None, padding="same")(uconv3)
        uconv3 = residual_block(uconv3,start_neurons * 8)
        uconv3 = residual_block(uconv3,start_neurons * 8)
        # [B, 32, 32, 128]
        uconv3 = tf.keras.layers.LeakyReLU(alpha=0.1)(uconv3)
        
        # [B, 64, 64, 64]
        deconv2 = tf.keras.layers.Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(uconv3)
        # [B, 64, 64, 32]
        conv2 = backbone.layers[65].output
        # [B, 64, 64, 96]
        uconv2 = concatenate([deconv2, conv2])
        uconv2 = tf.keras.layers.Dropout(0.1)(uconv2)
        
        # [B, 64, 64, 64]
        uconv2 = tf.keras.layers.Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(uconv2)
        uconv2 = residual_block(uconv2,start_neurons * 4)
        uconv2 = residual_block(uconv2,start_neurons * 4)
        uconv2 = tf.keras.layers.LeakyReLU(alpha=0.1)(uconv2)
        
        # [B, 128, 128, 32]
        deconv1 = tf.keras.layers.Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv2)
        # [B, 128, 128, 24]
        conv1 = backbone.layers[22].output
        # [B, 128, 128, 66]
        uconv1 = concatenate([deconv1, conv1])
        uconv1 = tf.keras.layers.Dropout(0.1)(uconv1)
        
        # [B, 128, 128, 32]
        uconv1 = tf.keras.layers.Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(uconv1)
        uconv1 = residual_block(uconv1,start_neurons * 2)
        uconv1 = residual_block(uconv1,start_neurons * 2)
        uconv1 = tf.keras.layers.LeakyReLU(alpha=0.1)(uconv1)
        
        # [B, 256, 256, 16]
        uconv0 = tf.keras.layers.Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv1)   
        uconv0 = tf.keras.layers.Dropout(0.1)(uconv0)
        uconv0 = tf.keras.layers.Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(uconv0)
        uconv0 = residual_block(uconv0,start_neurons * 1)
        uconv0 = residual_block(uconv0,start_neurons * 1)
        uconv0 = tf.keras.layers.LeakyReLU(alpha=0.1)(uconv0)

        uconv0 = tf.keras.layers.Dropout(dropout_rate/2)(uconv0)
        self.logits = tf.keras.layers.Conv2D(1, (1,1), padding="same", activation="sigmoid")(uconv0) 

In [4]:
tf.compat.v1.reset_default_graph()
model = Model()

Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [5]:
loss = tf.reduce_mean(bce_dice_loss(model.Y, model.logits))
loss

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


<tf.Tensor 'Mean_1:0' shape=() dtype=float32>

In [6]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [7]:
var_list = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
assignment_map, initialized_variable_names = utils_tf1.get_assignment_map_from_checkpoint(var_list, 
                                                                                          'out/model.ckpt')


INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = stem_conv/kernel:0, shape = (3, 3, 3, 48), *INIT_FROM_CKPT*
INFO:tensorflow:  name = stem_bn/gamma:0, shape = (48,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = stem_bn/beta:0, shape = (48,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = stem_bn/moving_mean:0, shape = (48,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = stem_bn/moving_variance:0, shape = (48,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = block1a_dwconv/depthwise_kernel:0, shape = (3, 3, 48, 1), *INIT_FROM_CKPT*
INFO:tensorflow:  name = block1a_bn/gamma:0, shape = (48,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = block1a_bn/beta:0, shape = (48,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = block1a_bn/moving_mean:0, shape = (48,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = block1a_bn/moving_variance:0, shape = (48,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = block1a_se_reduce/kernel:0, shape = (1, 1, 48, 12), *INIT_FROM_CKPT*
INFO:tensorflow:  name = block1a_se_redu

In [8]:
saver = tf.train.Saver(var_list = assignment_map)
saver.restore(sess, 'out/model.ckpt')

INFO:tensorflow:Restoring parameters from out/model.ckpt


In [11]:
def _parse_image_function(example_proto):
    image_feature_description = {
        "image": tf.compat.v1.FixedLenFeature([], tf.string),
        "mask": tf.compat.v1.FixedLenFeature([], tf.string),
    }
    features = tf.compat.v1.parse_single_example(example_proto, features=image_feature_description)
    image = tf.image.decode_png(features['image'], channels = 3)
    image = tf.image.resize(image, [512, 512])
    image = tf.cast(image, tf.float32) / 255.0
    features['image'] = image
    mask = tf.io.decode_raw(features['mask'], out_type="float")
    mask = tf.reshape(mask, [512, 512, 3])
    mask = tf.cast(mask, tf.float32)
    features['mask'] = mask
    return features

dataset_path = glob('content/gdrive/MyDrive/Dataset/*.tfrec')
raw_image_dataset = tf.data.TFRecordDataset(dataset_path)
dataset = raw_image_dataset.map(_parse_image_function)
dataset = dataset.make_one_shot_iterator().get_next()

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


In [19]:
r = sess.run(dataset)
mask = np.zeros_like(r['mask'])
mask[:,:,1] = 1
mask = cv2.bitwise_and(r['mask'], mask)[:,:,1:2]

In [22]:
r['image'].shape, mask.shape

((512, 512, 3), (512, 512, 1))

In [26]:
from skimage.transform import resize

img_resize = resize(r['image'], (256, 256), anti_aliasing=False)
mask_resize = resize(mask, (256, 256), anti_aliasing=False)

In [27]:
sess.run(loss, feed_dict = {model.X: [img_resize], model.Y: [mask_resize]})

1.5774696